In [ ]:
class VolStrategy(bt.SignalStrategy):
    params = (
        ('period1', 3),
        ('af', 0.005),
        ('afmax', 0.02),
        ('period4', 24),
        ('slPerc', 0.9)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        if opti == False:
            print('%s, %s' % (dt, txt))
        
    def notify_order(self, order):
        
        ordersdict.append(order)
        
        if order.status in [order.Submitted]:
            #self.log('ORDER SUBMITTED')
            return

        if order.status in [order.Accepted]:
            #self.log('ORDER ACCEPTED')
            return        
        
        if order.status in [order.Expired]:
            self.log('ORDER EXPIRED')
            
        if order.status in [order.Partial]:
            self.log('ORDER PARTIAL - order partially executed')
            
            
        if order.status in [order.Margin]:
            self.log('MARGIN - not enough cash to execute the order')
            #print('*' * 5, "MARGIN ORDER", self.datas[0].datetime.datetime(0))
                    
        if order.status in [order.Rejected]:
            self.log('REJECTED - order by broker')

        elif order.status in [order.Completed]:
            if order.isbuy():
#                 self.log(
#                     'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
#                     (order.executed.price,
#                      order.executed.value,
#                      order.executed.comm))
                #print('buy', order.executed.size)
    
                self.transac.append(('BUY', num2date(order.executed.dt), order.executed.size, order.executed.price, 
                                     order.executed.value, order.executed.pnl, order.executed.psize, 
                                     order.executed.pprice, self.broker.getvalue(), self.data.close[0]))
                transac2.append(('BUY', num2date(order.executed.dt), order.executed.size, order.executed.price, 
                                order.executed.value, order.executed.pnl, order.executed.psize, 
                                order.executed.pprice, self.broker.getvalue(), self.data.close[0]))

            else:  # Sell
#                 self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
#                          (order.executed.price,
#                           order.executed.value,
#                           order.executed.comm))
                #print('sell', order.executed.size)
    
                self.transac.append(('SELL', num2date(order.executed.dt), order.executed.size, order.executed.price, 
                                order.executed.value, order.executed.pnl, order.executed.psize, 
                                self.broker.getcash(), self.broker.getvalue(), self.data.close[0]))
                transac2.append(('SELL', num2date(order.executed.dt), order.executed.size, order.executed.price, 
                                order.executed.value, order.executed.pnl, order.executed.psize, 
                                self.broker.getcash(), self.broker.getvalue(), self.data.close[0]))

        # Sentinel to None: new orders allowed , num2date(trade.dtclose)
        self.order = None

        
    def notify_trade(self, trade):
        #self.log('***** OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        trades.append(('TRADE', num2date(trade.dtclose) if trade.dtclose > 0 else 0, trade.tradeid, trade.size, trade.price, 
                       trade.value, trade.pnl, trade.barlen, self.broker.getvalue(), self.data.close[0], 
                       trade.historyon, trade.history))
       
    def __init__(self):
        self.sl_order = None
        self.price_high = 0
        self.transac = []
        self.count = 0
        
        self.psar = bt.indicators.PSAR(self.data, period=self.params.period1, af=self.params.af, 
                                      afmax=self.params.afmax)
        #self.dmi = bt.indicators.DMI(self.data, period=self.params.period1)
        
        #self.sig = self.dmi - self.data
        
#         self.crossover = bt.indicators.CrossOver(self.dmi.psar, self.data.close)
        
#         self.sma1, self.sma2 = bt.indicators.SMA(self.data.open, period=3), bt.indicators.SMA(self.data, period=5)

#     def prenext(self):
#         pass
#         self.count += 1
#         print('PRE', self.datas[0].datetime.datetime(0), self.count)
        
      
        
        
    def next(self):
#         self.count += 1
#         print(self.datas[0].datetime.datetime(0), self.count, self.dmi[0], ' * ',self.sma1[0], self.sma2[0],
#               ' * ', self.data.open[0], self.data.close[0])

        
# Long startegy  :        
        if not self.position and self.psar < self.data.close[0] :
            self.buy()

        if self.position and self.psar >= self.data.close[0]:
            self.close()
        
# Short startegy  :      
#         if not self.position and self.psar > self.data.close[0] :
#             self.sell()

#         if self.position and self.psar <= self.data.close[0]:
#             self.close()
                
#         if self.sellsig and self.position:   
#                  self.close()
                
#             self.buyorder = self.sell()

        #self.signal_add(bt.SIGNAL_LONG, self.crossover)

#         if self.position:
#             self.price_high = self.data.close[0] if self.data.close[0] > self.price_high else self.price_high
#             self.price_sl_long = self.price_high * self.params.slPerc

#             # cancel existing stop loss and take profit orders
#             if self.sl_order:
#                 self.broker.cancel(self.sl_order)
#                 if log_sl == True: self.log('STOP LOSS CANCEL, Close: %.2f' % self.data.close[0])

#             self.sl_order = self.order_target_value(target=0.0, exectype=bt.Order.Stop, price=self.price_sl_long )
#             if log_sl == True: self.log('STOP LOSS CREATE, Close: %.2f, Stop loss price: %.2f' % (self.data.close[0], self.price_sl_long))
            
#         valida.append((self.datas[0].datetime.datetime(0), self.data.open[0], self.data.low[0], self.data.high[0], self.data.close[0], 
#                        self.position.size, self.position.price, self.buysig[0], self.sellsig[0], self.sma1[0], self.sma2[0],
#                        self.sma3[0], self.sma4[0], self.broker.getvalue(), len(self)))
        
    def stop(self):
        print('(Period1 %2d) (af %2d) (afmax %2d) (Comp %2d) Ending Value %2d - #trans %2d - %2d' %
                 (self.params.period1, self.params.af*1000, self.params.afmax*100, comp, 
                  self.broker.getvalue(), len(self.transac), len(transac2)))
              
        opti_runs.append((self.params.period1, self.params.af, self.params.afmax, self.params.period4, self.params.slPerc, comp, 
                          self.broker.getvalue(), len(self.transac), len(transac2)))